# Setup

In [1]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 290 (delta 168), reused 211 (delta 90), pack-reused 0 (from 0)
Receiving objects: 100% (290/290), 3.54 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (168/168), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [10]:
!uv sync --system


error: unexpected argument '--system' found

Usage: uv sync [OPTIONS]

For more information, try '--help'.


In [5]:
!uv pip install --system mlflow

Using Python 3.11.13 environment at: /usr
Resolved 79 packages in 615ms                                        
Prepared 12 packages in 467ms                                            
Uninstalled 2 packages in 14ms
Installed 12 packages in 275ms                              
 + databricks-sdk==0.74.0
 + flask-cors==6.0.2
 - google-auth==2.38.0
 + google-auth==2.45.0
 + graphene==3.4.3
 + graphql-core==3.2.7
 + graphql-relay==3.2.0
 + gunicorn==23.0.0
 + huey==2.5.5
 + mlflow==3.7.0
 + mlflow-skinny==3.7.0
 + mlflow-tracing==3.7.0
 - protobuf==3.20.3
 + protobuf==6.33.2


In [6]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version        Show the version and exit.
  --env-file PATH  Load environment variables from a dotenv file before
                   executing the command. Variables in the file will be loaded
                   but won't override existing environment variables.
  --help           Show this message and exit.

Commands:
  ai-commands  Manage MLflow AI commands for LLMs.
  artifacts    Upload, list, and download artifacts from an MLflow...
  autolog      Commands for autologging with MLflow.
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  mcp          Model Context Protocol (MCP) server for MLflow.
  models       Deploy MLflow models locally.
  run          Run an MLflow proj

In [4]:
!pip install flash-attn --no-build-isolation


  Using cached flash_attn-2.8.3.tar.gz (8.4 MB)
  Preparing metadata (setup.py) ... done
canceled
ERROR: Operation cancelled by user


In [ ]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

Resolved 69 packages in 231ms                                        ⠋ Resolving dependencies...                                                     
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0.36.0
    

# Run

In [ ]:
import torch

In [ ]:
!mlflow

/bin/bash: line 1: mlflow: command not found


In [4]:
import pandas as pd
df = pd.read_csv("data/data.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.csv'

NameError: name 'tokenizer' is not defined

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [34]:
import torch

text = "(attended): KhoaDACE190399 had attended this activity / Đào Anh Khoa đã tham gia hoạt động này (absent): KhoaDACE190399 had NOT attended this activity / Đào Anh Khoa đã vắng mặt buổi này (-): no data was given / chưa có dữ liệu"

tokens = tokenizer(text, return_tensors="pt")
print(tokens)

with torch.no_grad():
    output= model(**tokens)

output["logits"].shape

{'input_ids': tensor([[    0, 14157,  3148,  1187,  6374,  3995, 37272,    27,  8044,  1850,
         16311,  9594,  4917, 12627,  4424, 30498,  3148,  7104, 39573, 22304,
          3065, 20712,  8233,    95,  5483,   157,  3720,    14,  7520,  3931,
         11297,  2697,    23, 14157,  3377,  1302,  6502, 37272,    27,  8044,
          1850, 16311,  9594,  4917, 12627,  4424, 30498, 13986,  1085,  3148,
          7104, 39573, 22304,  3065, 20712,  8233,    95,  5483,   157,  3720,
            14,  3543,   373,   391,    23, 14157,   770, 37272,    27,  4397,
         16819, 26431,  2667,  1946,    95,   102,    10,  7015,  1934,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 

torch.Size([1, 80, 64001])